In [1]:
import pandas as pd
import time
import tqdm
import json
import os

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)

C:\Users\Admin\miniconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Полный справочник
df = pd.read_excel('okved2.xlsx', header = 2)
df = df[~df['Код'].str.contains('.', regex=False)]
df = df[df['Код'].str.contains('\d')]

C:\Users\Admin\miniconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


## Get actual codes

In [3]:
df_support = pd.read_excel('../../raw_data/Региональные меры поддержки Москва.xlsx')
df_land = pd.read_excel('../../raw_data/Помещения и сооружения.xlsx')
df_acc = pd.read_excel('../../raw_data/ОЭЗ и технопарки.xlsx')

In [4]:
okved_actual = []

for i in df_land['Перечень видов экономической деятельности, возможных к реализации на площадке']\
            .str.split(';').apply(lambda x: [i.split(' ')[0] for i in x]).values:
    okved_actual += i
    
for i in df_acc['Список отраслей'].str.split(';').apply(lambda x: [i.split(' ')[0] for i in x]).values:
    okved_actual += i

for i in df_support['ОКВЭД'].str.split(';').apply(lambda x: [i.split(' ')[0] for i in x]).values:
    okved_actual += i


In [5]:
okved_actual = sorted(set(okved_actual))
okved_actual = [i.replace('-', '') for i in okved_actual if i not in ('', 'Нет')]
len(okved_actual)

88

In [6]:
df = df[df['Код'].isin(okved_actual)]
df['Комментарий'] = df['Комментарий'].fillna('')
df['Комментарий'] = df['Комментарий'].str.split('Эта группировка не включает').str[0]
df['Комментарий'] = df['Комментарий'].str.replace('^', '', regex=False)
df = df.reset_index(drop=True)

# Gen desr

### Create collection

In [7]:
import chromadb
import sys
sys.path.append('../../')
from GigaChat import GigaChat
import dotenv
dotenv.load_dotenv()

True

In [8]:
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="okved_collection")

### Create additional descroptions

In [10]:
# create YandexGPT instance
auth_token = os.environ.get("AUTH_TOKEN")

gc = GigaChat(auth_token = auth_token)


New iam token was generated


In [11]:
prompts = {'system_prompt_1': 'Перефразируй указанный текст, раскрой его смысл и укажи дополнительные детали',
           'system_prompt_2': 'Раскрой смысл данной экономической деятельность - опиши и приведи примеры',
           'system_prompt_3': 'Сделай расширенное описание по указанному тексту с экономической деятельность',
           'original': None
          }

df_embs = []

for _, i in tqdm.tqdm(df.iterrows(), total = len(df)):
    for prompt_id, system_prompt in prompts.items():
        if prompt_id != 'original':
            orig_desc = i['Наименование'] + '\n' + i['Комментарий']
            desc = gc.generate(system_prompt=system_prompt, user_prompt=orig_desc, model='GigaChat')
        else:
            desc = orig_desc
        time.sleep(0.1)
        df_embs.append({'code': i['Код'],
                        'text': desc,
                        'text_type': prompt_id,
                        })
        
        
df_embs = pd.DataFrame(df_embs)
df_embs.to_hdf('df_okved.h5', key='df')

  0%|                                                                                           | 0/88 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [79]:
if 'emb' not in df_embs.columns:
    df_embs['emb'] = None
    
embs_vals = {}
for n, i in tqdm.tqdm(df_embs[df_embs['emb'].isna()].iterrows(), total = len(df_embs)):
    try:
        embs_vals[n] = gc.get_embedding(i['text'])[0]
        time.sleep(0.05)
    except:
        print(n)


 39%|███████████████████████████████▌                                                | 139/352 [01:40<01:57,  1.82it/s]

<module 'requests' from 'C:\\Users\\Admin\\miniconda3\\lib\\site-packages\\requests\\__init__.py'>
<Response [413]>
138


100%|████████████████████████████████████████████████████████████████████████████████| 352/352 [04:14<00:00,  1.38it/s]


In [102]:
df_embs.loc[list(embs_vals.keys()), 'emb'] = pd.Series(list(embs_vals.values()), index = list(embs_vals.keys()), )

In [106]:
df_embs = df_embs[df_embs['emb'].notna()]

In [107]:
df_embs.shape

(351, 4)

In [108]:
df_embs.to_hdf('df_okved_with_embs.h5', key='df')

### Add Data to collection

In [112]:
df_embs

,code,text,text_type,emb
0,01,Эта категория объединяет два основных направле...,system_prompt_1,"[-0.1944189965724945, -0.7687526941299438, -0...."
1,01,Эта экономическая деятельность включает в себя...,system_prompt_2,"[0.0010591903701424599, -0.8907296657562256, -..."
2,01,Эта категория включает в себя два основных вид...,system_prompt_3,"[-0.2407291680574417, -0.7296691536903381, -0...."
3,01,"Растениеводство и животноводство, охота и пред...",original,"[0.25101834535598755, -1.0408835411071777, -0...."
4,02,"Лесоводство и лесозаготовка - это отрасль, кот...",system_prompt_1,"[0.03224073350429535, -0.017395544797182083, -..."
5,02,Лесоводство и лесозаготовки - это экономическа...,system_prompt_2,"[0.5577157139778137, 0.14201781153678894, -0.4..."
6,02,Лесоводство и лесозаготовки - это отрасль экон...,system_prompt_3,"[0.5026388168334961, -0.5149738788604736, -0.6..."
7,02,Лесоводство и лесозаготовки\nЭта группировка в...,original,"[0.30898576974868774, -0.16251832246780396, -0..."
8,03,Эта группа объединяет два вида деятельности: р...,system_prompt_1,"[0.24928432703018188, -0.5609805583953857, -0...."
9,03,Рыболовство и рыбоводство - это экономическая ...,system_prompt_2,"[0.2513660490512848, -0.8031306266784668, -0.5..."


In [117]:
for n, i in df_embs.iterrows():
    collection.add(documents=[i['text']],
                            ids=[str(n).zfill(3)],
                            embeddings=[i['emb']],
                            metadatas=[{"code": i["code"]}])

In [120]:
results = collection.query(
    query_embeddings=gc.get_embedding("Я разрабатываю сервисные приложения"),
    n_results=10
)
print(results)

New iam token was generated
{'ids': [['218', '216', '131', '221', '219', '130', '128', '222', '217', '197']], 'distances': [[283.0162048339844, 284.30560302734375, 285.7375793457031, 286.0849609375, 288.27740478515625, 288.30206298828125, 289.2582092285156, 289.5389404296875, 290.24847412109375, 296.23626708984375]], 'metadatas': [[{'code': '62'}, {'code': '62'}, {'code': '35'}, {'code': '63'}, {'code': '62'}, {'code': '35'}, {'code': '35'}, {'code': '63'}, {'code': '62'}, {'code': '56'}]], 'embeddings': None, 'documents': [['Эта группа включает в себя широкий спектр услуг, связанных с разработкой и поддержкой компьютерного программного обеспечения. Сюда входят такие виды деятельности, как проведение экспертиз в области информационных технологий, разработка, изменение и поддержка программного обеспечения, планирование и проектирование компьютерных систем, объединяющих компьютерное оборудование, программное обеспечение и коммуникационные технологии. Также сюда входит интерактивное управ